In [ ]:
%cd ~
import pyfits,glob,time,scipy
import scipy.interpolate
from numpy import *
from matplotlib.pyplot import *
from subprocess import call
from IPython import parallel

thedir='/sandbox/lsst/lsst/GUI/testdata/'
%cd $thedir
configfile='/sandbox/lsst/lsst/GUI/sextractor/default-array_dither.sex'
paramfile='/sandbox/lsst/lsst/GUI/sextractor/default-array_dither.param'
maskdir='/sandbox/lsst/lsst/GUI/sextractor/masks/'

%matplotlib inline

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return finalimg

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    thedir='/sandbox/lsst/lsst/GUI/testdata/'
    %cd $thedir
    configfile='/sandbox/lsst/lsst/GUI/sextractor/default-array_dither.sex'
    paramfile='/sandbox/lsst/lsst/GUI/sextractor/default-array_dither.param'
    maskdir='/sandbox/lsst/lsst/GUI/sextractor/masks/'
    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'114-04_spot-30um_light_1??_20150630??????.fits'))

In [ ]:
zfilelist

In [ ]:
zfilelist[0][-23:-20]

In [ ]:
#### This cell runs sextractor on all files in the z file list in PARALLEL
clients=parallel.Client()
clients.block=True
print clients.ids
view=clients.load_balanced_view()
view.map(ovsub_runsex_makereg,zfilelist)

#### This cell runs sextractor on all files in the z file list in SERIAL
for fitsfilename in zfilelist:
    tfile1=time.time()
    segnums=[1,4,8,14]
    for i in range(1,17):
        tstart=time.time()
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')
        tend=time.time()
        print extname+" time: "+str(tend-tstart)[:4]
    print "Time taken for file "+str(fitsfilename[-23:-20])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
nx,ny =10,40 # number of grid divs per segment (4x more in y than in x)

measname='A_IMAGE'   # the measurement type that you want to map (must be in paramfile defined at top)
for fitsfilename in zfilelist[23:-7]:
    allx,ally,allmeas=[],[],[]
    figure()
    for i in range(1,17):
        hdr=pyfits.getheader(fitsfilename,i)
        extname=hdr['EXTNAME']
        catname=fitsfilename[:-5]+extname+'.fits.cat'
        cat=pyfits.getdata(catname,'LDAC_OBJECTS')
        good=where((cat['FLAGS']==0) & (cat['A_IMAGE']<5) & (cat['MAGERR_AUTO']<.025))
        xcoo,ycoo=cat['XWIN_IMAGE'][good],cat['YWIN_IMAGE'][good]
        segx,segy=xcoo+offsetdict[extname][0],offsetdict[extname][2]*ycoo+offsetdict[extname][1]
        segmeas=cat[measname][good]
        allx.extend(segx)   #append individual segment coordinates as global ones
        ally.extend(segy)   #y segment coordinates increase toward mid
        allmeas.extend(segmeas)      # append the measurement to the list
        segxgrid,segygrid=np.meshgrid(np.linspace(min(segx),max(segx),nx),np.linspace(min(segy),max(segy),ny))
        segmeasgrid=scipy.interpolate.griddata((segx,segy),segmeas,(segxgrid,segygrid),method='linear')
        pcolor(segxgrid,segygrid,segmeasgrid,vmin=1,vmax=1.4,cmap=cm.gist_heat_r)
    #xgrid,ygrid=np.meshgrid(np.linspace(0,4000,nx*8),np.linspace(0,4000,ny*2))
    #meas_grid=scipy.interpolate.griddata((allx,ally),allmeas,(xgrid,ygrid),method='linear')
    #pcolor(xgrid,ygrid,meas_grid,vmin=1,vmax=3,cmap=cm.gist_heat_r)
    axis([0,4000,0,4000])
    title("Measurement: "+measname+", Location: z="+str(pyfits.open(fitsfilename)[17].header['Z_POS'])+"$\mu m$")
    colorbar()
    savefig(fitsfilename[-23:-20]+'-'+measname+'.png',dpi=200)

In [ ]:
catname

In [ ]:
plot(cat['MAG_AUTO'],cat['MAGERR_AUTO'],'k.')

In [ ]:
ycoo.max()